#Recolección datos M2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install pygeohash
!pip install geopandas
!pip install polygeohasher
!pip install contextily
!pip install matplotlib
!pip install shapely
!pip install gmaps
!pip install ipywidgets
!pip install widgetsnbextension
!pip install pysal

In [0]:
from shapely.geometry import Polygon
from geopandas import GeoSeries
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from polygeohasher import polygeohasher

def MallaCaptura(poligono):
  #toma el poligono y lo convierte en una Geoserie
  g = GeoSeries(Bta)
  #toma la geoserio y la convierte en Geo data frame
  gdf = gpd.GeoDataFrame(geometry=g)
  # Tamaño de celda
  # https://www.geospatialworld.net/blogs/polygeohasher-an-optimized-way-to-create-geohashes/
  primary_df = polygeohasher.create_geohash_list(gdf, 5,inner=False)
  geo_df = polygeohasher.geohashes_to_geometry(primary_df,"geohash_list").set_crs('epsg:4326')
  # Asigna las coordenadas del geohash a cada vertice
  geohash_bta = pd.concat([geo_df,geo_df.geometry.bounds], axis=1)
  return geohash_bta


In [0]:
Bta = Polygon([(-74.24835205078124, 4.468272728744618),(-73.99497985839844, 4.468272728744618),(-73.99497985839844, 4.837154404509952),(-74.24835205078124,4.837154404509952)])
Malla = MallaCaptura(Bta)
#.explore(height='70')

In [0]:
Malla.explore()

In [0]:
import numpy as np

text_query='{{"queries":[{"types":["propertiesByFiltersQuery"],"filter":{"propertyTypeId":{"values":["1"]},"businessTypeId":{"values":["2"]},"status":{"values":["Usado"]},"priceRange":{"values":["0","100000000000000000"]},"geoShape":{"values":%s}}}]}'


Malla['geoShape']=Malla['geometry'].apply(lambda x: text_query %(str([list(map(str, punto[::-1])) for punto in np.dstack(x.boundary.coords.xy).tolist()[0]])))


In [0]:
for a in Malla['geoShape'].values.tolist():
  print(a.replace("'",'"'))

In [0]:
!pip install requests-oauthlib

In [0]:
import requests
import json

from requests.auth import HTTPBasicAuth


url='https://commons-api.metrocuadrado.com/v1/api/commons/queries?content-type=application/json'

headers = {'x-api-key':'6JgwwXGxlC921DP4SB4ST6Jo6OO7rv3t4yXn5Y8y'}


dic = json.loads('{"queries":[{"types":["propertiesByFiltersQuery"],"filter":{"neighborhoodId":{"values":["0001-1384"]},"propertyTypeId":{"values":["1"]},"businessTypeId":{"values":["2"]},"status":{"values":["Usado"]},"priceRange":{"values":["0","100000000000000000"]},"geoShape":{"values":[["4.679023840322769","-74.08829825950835"],["4.675002226534648","-74.09065784909697"],["4.67583649937833","-74.08744022693068"]]}}}]}')


requests.get(url, data=headers)

In [0]:




text = text_query % ()